#### Guided Project: Building a spam filter using the Naive Bayes Theorem
The goad of the project is to build a spam filder using the multinomail Naive Bayes algorithm. 

The naive Bayes learn from emails the human has already classified as spam. It can be sum up as 
-1- learn how human classify messages
-2- use this knowledge to compute / estimate some proba (spam and non-spam)
-3- apply these generated rules to new messages

the data we gonna use as input to build the spam fuild is a datasset of 5572 sms that are already labeled by a human. :
- dataset from Tiago A. Almeida and José María Gómez Hidalgo
- link :https://archive.ics.uci.edu/ml/datasets/sms+spam+collection
- the method to aggregate the data http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition


### Exploring the dataset
Let's get familiar with the dataset we gonna use the libray Pandas

In [94]:
import pandas as pd

In [95]:
sms=pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label','SMS'])
#sms.name = 'All dataset'

In [96]:
sms['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

we see that SMS were classified either as ham or spam :)
- 4825 non_spam messages
- 747spams
so we can already compute some probabilities !

In [97]:
def ratio_spam_no_spam(df):
    n_non_spam= df['Label'].value_counts()[0]
    n_spam= df['Label'].value_counts()[1]
    total_sms= df['Label'].count()

    p_non_spam = n_non_spam / total_sms
    p_spam = n_spam / total_sms

    return print("{0:.2f}% of the {1} is spam and {2:.2f}% is non_spam".format(p_spam*100,df.name,p_non_spam*100))

In [98]:
#ratio_spam_no_spam(sms)

### Splitting test / training set 
In order to validate the rules we gonna create form these messages we gonna spit the dataset between
- a training set : will be use to generate the rules
- a test set : will be used to test the rules we generated

We gonna split 4/5 for the training and 1/5 for the test set meaning 
- 4458 messages for training set 
- 1114 messages for testing 
1114 + 4458 = 5572 :D

inthe next blocs we gonna create the training set and use the .drop method to attriute the remaining sns to the test set

In [99]:
#creating the training set 
training_set=sms.sample(frac=0.8,random_state=1)
training_set.name='Training Set'

In [100]:
training_set.sample(3,random_state=1)

,Label,SMS
224,ham,U say leh... Of course nothing happen lar. Not...
4527,spam,"I want some cock! My hubby's away, I need a re..."
3201,ham,Just curious because my cuz asked what I was u...


In [101]:
training_set.count()

Label    4458
SMS      4458
dtype: int64

all seeems fine

In [102]:
#creating test set
test_set= sms.drop(training_set.index)
test_set.name='Test Set'

In [103]:
test_set

,Label,SMS
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
15,spam,"XXXMobileMovieClub: To use your credit, click ..."
18,ham,Fine if thats the way u feel. Thats the way ...
20,ham,Is that seriously how you spell his name?
...,...,...
5546,ham,Ic. There are a lotta childporn cars then.
5549,ham,"You know, wot people wear. T shirts, jumpers, ..."
5552,ham,Have a safe trip to Nigeria. Wish you happines...
5555,ham,Yeh. Indians was nice. Tho it did kane me off ...


In [104]:
#test if one index of the training set inthe the test test 
#test_set.loc[4527] # index belonging to the training test

we get an error message all fine. 

let's check the ration spam / non spam is both sets

In [105]:
#ratio_spam_no_spam(sms)

In [106]:
#ratio_spam_no_spam(training_set)

In [107]:
#ratio_spam_no_spam(test_set)

The ratios between the different data sets are similar we can continue.

## Letter case and punctuation
in order to create a db of vocabulary we gonna remove all punctuation and lowercase all

In [108]:
#fixing the SMS columns
training_set['SMS']=training_set['SMS'].str.replace('\W', ' ') #regex for any characters which is not A-Z a-z 0-9
training_set['SMS']=training_set['SMS'].str.lower() #to have all in lowercase
training_set.head()

,Label,SMS
1078,ham,yep by the pretty sculpture
4028,ham,yes princess are you going to make me moan
958,ham,welp apparently he retired
4642,ham,havent
4674,ham,i forgot 2 ask ü all smth there s a card on ...


### Creation of the vocabulary
- we gonna loop through all the sms and
- each sms split by words
- each words add to list 
- list transformed to set to remove dupli then back to a list 

##  !!!!! 
because of the approach in counting the words, the index of the training and test sets need to be reset, when I will do the approach with the fingerprinting it won't necessary. 


In [109]:
#reset index
training_set= training_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

In [110]:
training_set['SMS']=training_set['SMS'].str.split()
vocabulary = [] #the list
for text in training_set['SMS']: #loop through sms
    for word in text:
        vocabulary.append(word)

In [111]:
#how big is our vocabulary
print(len(vocabulary))#list to set to remove duplicate
vocabulary = list(set(vocabulary))
print(len(vocabulary))

72427
7783


- We effectively removed a lot duplicated words

### Final training set
now we have our vocabulary, we gonna count 
transform each sms to a fingerprint as counts of words appear from our vocabulary databases

we first create a 
see block down for an exemple

In [112]:
#exemple
'''

word_counts_per_sms_exemple = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }
word_counts_exemple = pd.DataFrame(word_counts_per_sms_exemple)
word_counts_exemple.head()
'''

"\n\nword_counts_per_sms_exemple = {'secret': [2,1,1],\n                       'prize': [2,0,1],\n                       'claim': [1,0,1],\n                       'now': [1,0,1],\n                       'coming': [0,1,0],\n                       'to': [0,1,0],\n                       'my': [0,1,0],\n                       'party': [0,1,0],\n                       'winner': [0,0,1]\n                      }\nword_counts_exemple = pd.DataFrame(word_counts_per_sms_exemple)\nword_counts_exemple.head()\n"

In [113]:
#initation the dictionnary with each value is a array of the size of the size of the training set
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}
for index, text in enumerate(training_set['SMS']):
    for word in text:
        word_counts_per_sms[word][index] += 1

In [114]:
#dictionnary to DataFrame
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,waves,bac,congrats,3100,delhi,dessert,toppoly,hurting,crave,kid,...,prey,dine,totes,proverb,place,ph,collapsed,welp,hand,yahoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
training_set_clean=pd.concat([training_set,word_counts],axis='columns')
training_set_clean.head()

,Label,SMS,waves,bac,congrats,3100,delhi,dessert,toppoly,hurting,...,prey,dine,totes,proverb,place,ph,collapsed,welp,hand,yahoo
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We have now all the info needed to start computing some proba

### Computing the constants
Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

- p_spam and p_ham
- n_Spam, n_Ham, n_vocabulary
- Recall that:

    - n_spam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.
    - n_ham is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.
    - We'll also use Laplace smoothing and set . If you need help at any point, don't forget that you can find the solutions notebook here.

In [116]:
#number of words in the spam messages 
spam_training_set = training_set_clean[training_set_clean['Label']=='spam']
ham_training_set=training_set_clean[training_set_clean['Label']=='ham']

# p(spam) and P(ham)
p_spam = len(spam_training_set) / len(training_set_clean)
p_ham = len(ham_training_set) / len(training_set_clean)

#N_spam
n_spam = spam_training_set.apply(len).sum()
#N Ham
n_ham = ham_training_set.apply(len).sum()

#N_vocabulary
n_vocabulary= len(vocabulary)

#Laplace smoothing 
alpha=1

### Calculation parameters

- in order to compute P(wi | spam) et P(wi|ham) on va creer 2 dictionnairies, one for spam and one for ham, the dictionnary with the vocabulary databse we create above 
- then we loop through the spammessages and compute the proba P(wi|spam) with 
        P(wi|spam) = ( N(wi|spam) + alpha) / (Nspam + N_vocabulary * alpha)
- same with ham messages 

In [117]:
#set up the dictionnaries:
vocabulary_spam={}
vocabulary_ham={}
for word in vocabulary:
    vocabulary_spam[word]=0
    vocabulary_ham[word]=0

In [118]:
#compute proba wi|spam
for key in vocabulary_spam:
    vocabulary_spam[key]=(spam_training_set[key].sum() + alpha) / ( n_spam + n_vocabulary * alpha)
    
#proba wi|ham
for key in vocabulary_ham:
     vocabulary_ham[key]=(ham_training_set[key].sum() + alpha) / ( n_ham + n_vocabulary * alpha)

In [119]:
print(vocabulary_spam['welp'])
print(vocabulary_ham['welp'])

2.1373079281514018e-07
1.3314554042493331e-07


 better to call the dictionnaries parameters

In [120]:
parameters_spam= vocabulary_spam
parameters_ham = vocabulary_ham

### Classifiying a new message
now that we have computed the proba of each words if a spam or an ham message, if we take a new message we can then compute for this message:
- P(Spam | w1,w2...wn) and - P(Ham | w1,w2...wn) meaning the proba the message is either a spam or a ham while knowing its content (and this based on the rules derived from the training set).

we gonna right all that as a function that  :
- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

=> for that we gonna use the test set :D 

In [121]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower().split()
        
#compute the p_spam_given message by looping through the message and updating the probabily
    p_spam_given_message = p_spam 
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message = p_spam_given_message * parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message = p_ham_given_message * parameters_ham[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

test cases of our function
- 'WINNER!! This is the secret code to unlock the money: C3421.'
- "Sounds good, Tom, then see u there"

In [122]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 2.2361181244033363e-46
P(Ham|message): 2.0179400655053473e-51
Label: Spam


In [123]:
classify(" dinner at my place")

P(Spam|message): 3.0332159335259197e-25
P(Ham|message): 2.6170449987544287e-22
Label: Ham


In [128]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 1.676839627472796e-41
P(Ham|message): 8.202208476864172e-40
Label: Ham


### Accuracy of our new spam filter


In [124]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [125]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

In [126]:
test_set.head()

,Label,SMS,predicted
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,spam
1,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,spam
2,spam,"XXXMobileMovieClub: To use your credit, click ...",spam
3,ham,Fine if thats the way u feel. Thats the way ...,ham
4,ham,Is that seriously how you spell his name?,ham


In [127]:
#measure accurary
correct=0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 999
Incorrect: 115
Accuracy: 0.8967684021543986


## Conclusion
We created a spam filer who management to sorted out correctly almost 90% of all emails of the test set. 


## Going further
- Isolate the 115 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.
- based on the énoncé, we should had like 98% accurary, we should check why we have less. 
- Get the project portfolio-ready by using a few tips from our style guide for data science projects.

solution:https://github.com/dataquestio/solutions/blob/master/Mission433Solutions.ipynb